<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Tipologia i cicle de vida de les dades · <b>Pràctica 2</b></p>
<p style="margin: 0; text-align:right;">Màster universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis de Informàtica, Multimèdia i Telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Pràctica 2: Neteja i anàlisi de les dades

### **Autors:** Andrés Laverde Marín i Josep Mª Espasa Verdés

Aquesta pràctica es basarà en el tractament del dataset [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic).

### Índex

<ol start="1">
  <li>Descripció del dataset (Jo faria una petita introducció aqui, si vols la podem desenvolupar en detall al document)</li>
  <li>Integració i selecció de les dades d'interès a analitzar</li>
  <li>Neteja de les dades</li>
  <li>Anàlisi de les dades</li>
  <li>Representació dels resultats</li>
  <li>Resolució del problema</li>
</ol>

In [1]:
# Carrega les llibreries necessàries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# 1. Descripció del dataset 

El dataset seleccionat ha estat [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic) que ens proporciona *kaggle* amb l'objectiu de determinar quins dels pasatgers del titanic sobreviuen a [l'accident del 15 d'abril de 1912](https://es.wikipedia.org/wiki/RMS_Titanic) en el qual van morir 1514 persones de les 2223 que viatjaven en el transatlàntic.

L'objectiu de la pràctica és entendre i predir quins grups de persones tenen més probabilitats de sobreviure, encara que sabem que hi ha un component de sort en la supervivència de les persones. Per dur a terme aquest objectiu partim de dos conjunts de dades, un que ens servirà com a entrenament, el qual compte amb 891 registres, i l'altre com test, el qual compte amb 418 registres, per validar els algoritmes de predicció fets servir.

# 2. Intregració i selecció de les dades d'interès a analitzar

Primerament es carregen les dades dels fitxers de partida (format CSV) en objectes DataFrames de Pandas, per poder realitzar les diferents anàlisis.

In [2]:
# Carrega els fitxers de dades (entrenament i test) en dos dataframes
titanic_train_raw = pd.read_csv(os.getcwd()+'/data/in/train.csv')
titanic_test_raw = pd.read_csv(os.getcwd()+'/data/in/test.csv')

A mode d'exploració inicial, es mostra les primeres files dels DataFrames i es comprova que la tipologia dels diferents atributs carregats és la correcta.

In [3]:
# Mostra les primeres files de les dades carregades
titanic_train_raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Comprovacio de la tipologia de les dades carregades
titanic_train_raw.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [5]:
titanic_test_raw.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
titanic_test_raw.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

De tots els atributs recollits en el DataSet, les dades de nom (*Name*), número de tiquet (*Ticket*) i número de cabina (*Cabin*) són atributs que, per la seva pròpia naturalessa, no es poden considerar, com factors que poden afectar la supervivència o no d'un determinat passatger i, per tant, no es seleccionaran les columnes associades a aquests atributs. A continuació es realitza aquesta selecció.

In [7]:
# Elimina les columnes 'Name' i 'Ticket' dels DataFrames d'entrenament i test
titanic_train = titanic_train_raw.drop(columns=['Name', 'Ticket', 'Cabin'])
titanic_test = titanic_test_raw.drop(columns=['Name', 'Ticket', 'Cabin'])

# 3. Neteja de les dades

### 3.1 Les dades contenen zeros o elements buits? Com gestionaries aquests casos?

### Anàlisi zeros

Per cercar els atributs que contenen zeros, s'utilitza el mètode ```describe``` de pandas que mostra un resum estadístic de les variables quantitatives, així, entre les dades mostrades, hi ha els valors mínims que, en cas de contenir zeros, seran els valors mínims.

In [8]:
# Mostra un resum estadistic dels DataFrames d'entrenament i test
titanic_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
titanic_test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


Tal com es pot observar en la sortida de dades, els camps que contenen zeros són la supervivència (*Survived*), el nombre de germans o cònjuge a bord del Titànic (*Sibsp*), el nombre de pares o nens a bord del Titànic (*Parch*) i el preu del bitllet (*Fare*).

De tots ells, són totalment lògics i legítims els valors zero en el atributs *Survived, Sibsp* i *Parch*, en canvi, un preu de bitllet igual a zero, inicialment, no resulta lògic, més enllà de considerar que aquests passatgers com pertanyents a la tripulació, segurament la hipòtesis més raonable. Després d'una recerca per les xarxes amb els noms de les persones que acompleixen aquesta condició es dóna com a raonable aquest fet i s'accepten com a legítims, també, aquests valors de zero pel preu de bitllet i, per tant, no es relitzarà cap tractament al respecte.

### Anàlisi valors nuls

In [10]:
# Comprova el nombre de dades amb valors nuls en les dades d'entrenament i de test
titanic_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

In [11]:
titanic_test.isnull().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
Embarked        0
dtype: int64

Tal com es pot veure en la sortida de dades, existeixen valors nuls en l'atribut d'edat (*Age*) en els dos DataSets, tant el d'entrenament com el de test, i, en el cas del conjunt de dades d'entrenament, també presenta valors nuls el camp del port d'embarcament (*Embarked*), en dos instàncies, mentre en el cas del conjunt de dades de test, presenta un valor nul el camp del preu del bitllet (*Fare*).

El tractament d'aquests valors nuls es realitza de manera diferencial segons l'atribut considerat, així, els nuls en els camps de port d'embarcament (*Embarked*) i preu del bitllet (*Fare*), donat que tenen una freqüència molt baixa, s'intentaran corregir manualment, gràcies a la gran quantitat d'informació disponible per la xarxa, referent al viatge del Titànci. En canvi, el tractament dels valors nuls de l'atribut d'edat (*Age*), que presenta una freqüència important (177 i 86 instàncies en el conjunt de dades d'entrenament i de test, respectivament) no pot plantejar-se la correcció manual i, per tant, es planteja algun altre tipus de tractament que impliqui l'aplicació d'un mètode general. Així, una primera opció seria eliminar aquests registres nuls, però aquesta opció es descarta degut a la important pèrdua d'informació que representa, per tant, s'opta per emprar un mètode d'imputació de dades i, d'entre els disponibles, s'elegeix el missForest, ja que és considerat un mètode robust que, alhora, permet treballar amb dades mixtes.

Seguidament es realitza la imputació manual dels valors nuls dels atributs de port d'embarcament (*Embarked*) i preu del bitllet (*Fare*).

**Imputació manual -> TODO**

In [12]:
# Mostra els valors nuls del dataframe d'entrenament
titanic_train_raw[titanic_train_raw['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


Tal com es recull en l'[article referent a Mrs Martha Evelyn Stone](https://www.encyclopedia-titanica.org/titanic-survivor/martha-evelyn-stone.html) de l'enciclopèdia titànica, aquestes passatgeres van embarcar al Titànic al port de Southampot, per tant, els correspon un valor d''*S*' en el camp '*Embarked*'. A continuació es realitza aquesta imputació.

In [13]:
# Imputacio manual del port d'embarcament
titanic_train.loc[titanic_train['PassengerId']==62, ['Embarked']]='S'
titanic_train.loc[titanic_train['PassengerId']==830, ['Embarked']]='S'

In [14]:
# Mostra els valors nuls del dataframe de test
titanic_test_raw[titanic_test_raw['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


Tal com es recull en l'[article referent a Mr Thomas Storey](https://www.encyclopedia-titanica.org/titanic-victim/thomas-storey.html) de l'enciclopèdia titànica, aquest passatger es pot considerar tripulació i, per tant, se l'imputarà un preu de bitllet igual a zero. A continuació es realitza aquesta imputació.

In [15]:
# Imputacio manual del port d'embarcament
titanic_test.loc[titanic_test['PassengerId']==1044, ['Fare']]=0

A continuació s'imputen els valors nuls d'edat emprant el mètode missForest, cal comentar que s'utilitza la implementació de la llibreria ```missingpy``` que, en cas de no estar instal.lada, es pot instal·lar mitjançant la comanda ```pip install missingpy```, per a més informació al respecte, es pot consultar la [documentació de la llibreria missingpy](https://pypi.org/project/missingpy/). D'altra banda, també és important indicar que les variables categòriques s'han recodificat com vectors de tipus one-hot, per tal de poder ser interpretades correctament.

In [16]:
# Preparació de les dades categoriques com vectors one-hot
titanic_train = pd.get_dummies(titanic_train, ['Sex', 'Embarked'])
titanic_test = pd.get_dummies(titanic_test, ['Sex', 'Embarked'])

# Carrega la llibreria que conte la implementacio missForest
from missingpy import MissForest

# Imputa els valors nuls emprant el metode missForest tot indicant les columnes de valors categorics
imputer = MissForest()
X_imputed_train = imputer.fit_transform(
                    titanic_train, cat_vars=[titanic_train.columns.get_loc('Sex_female'),
                                        titanic_train.columns.get_loc('Sex_male'),
                                        titanic_train.columns.get_loc('Embarked_C'),
                                        titanic_train.columns.get_loc('Embarked_Q'),
                                        titanic_train.columns.get_loc('Embarked_S')]
                    )

X_imputed_test = imputer.fit_transform(
                    titanic_test, cat_vars=[titanic_test.columns.get_loc('Sex_female'),
                                        titanic_test.columns.get_loc('Sex_male'),
                                        titanic_test.columns.get_loc('Embarked_C'),
                                        titanic_test.columns.get_loc('Embarked_Q'),
                                        titanic_test.columns.get_loc('Embarked_S')]
                    )


# Converteix la matriu de sortida en un DataFrame
titanic_train_final = pd.DataFrame(X_imputed_train, columns=titanic_train.columns.tolist())
titanic_test_final = pd.DataFrame(X_imputed_test, columns=titanic_test.columns.tolist())

/Applications/anaconda3/lib/python3.7/site-packages/missingpy/missforest.py:407: RuntimeWarning: invalid value encountered in long_scalars
  (Ximp[:, self.cat_vars_] != Ximp_old[:, self.cat_vars_])) / n_catmissing


Iteration: 0


/Applications/anaconda3/lib/python3.7/site-packages/missingpy/missforest.py:407: RuntimeWarning: invalid value encountered in long_scalars
  (Ximp[:, self.cat_vars_] != Ximp_old[:, self.cat_vars_])) / n_catmissing


Iteration: 1


/Applications/anaconda3/lib/python3.7/site-packages/missingpy/missforest.py:407: RuntimeWarning: invalid value encountered in long_scalars
  (Ximp[:, self.cat_vars_] != Ximp_old[:, self.cat_vars_])) / n_catmissing


...

/Applications/anaconda3/lib/python3.7/site-packages/missingpy/missforest.py:407: RuntimeWarning: invalid value encountered in long_scalars
  (Ximp[:, self.cat_vars_] != Ximp_old[:, self.cat_vars_])) / n_catmissing


Iteration: 0


/Applications/anaconda3/lib/python3.7/site-packages/missingpy/missforest.py:407: RuntimeWarning: invalid value encountered in long_scalars
  (Ximp[:, self.cat_vars_] != Ximp_old[:, self.cat_vars_])) / n_catmissing


Iteration: 1
Iteration: 2


/Applications/anaconda3/lib/python3.7/site-packages/missingpy/missforest.py:407: RuntimeWarning: invalid value encountered in long_scalars
  (Ximp[:, self.cat_vars_] != Ximp_old[:, self.cat_vars_])) / n_catmissing


***

# Per esborrar / fer proves

In [17]:
titanic_train_final.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1.0,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,1.0
1,2.0,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0
2,3.0,1.0,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,1.0
3,4.0,1.0,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,1.0
4,5.0,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0


In [18]:
titanic_train[titanic_train['Embarked'].isnull()]

KeyError: 'Embarked'

In [ ]:
titanic_test[titanic_test['Fare'].isnull()]

In [ ]:
titanic_test[titanic_test['Cabin'].isnull()].describe()

In [ ]:
titanic_train[titanic_train['Fare']==0]

In [ ]:
titanic_test[titanic_test['Fare']==0]

In [ ]:
titanic_test.head()

In [ ]:
titanic_train[titanic_train['Fare']==0]

In [ ]:
titanic_train.apply(lambda x: x)

In [ ]:
titanic_train[titanic_train['Cabin'].isnull()].describe()

In [ ]:
titanic_train.describe()

In [ ]:
titanic_train.head()

In [ ]:
titanic_train.dtypes

In [ ]:
titanic_train.isnull().sum()

In [ ]:
titanic_test.isnull().sum()

In [ ]:
titanic_train[titanic_train['Embarked'].isnull()]